In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage


In [23]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("start-maximized")
# chrome_options.add_argument("disable-infobars")
# chrome_options.add_argument("--disable-extensions")

#crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
# chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
#webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
#套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
driver.get("https://waste.epa.gov.tw/prog/IndexFrame.asp?Func=2")
# driver.s
driver.set_window_size(1050,840)

driver.save_screenshot(r"D:\trydata\screenshot.png")
driver.switch_to_default_content()
driver.switch_to_frame(1)
driver.page_source
accountInput=driver.find_element_by_name("fac_user")
accountInput.send_keys("A45B1625")

pwInput=driver.find_element_by_name("fac_pwd")
pwInput.send_keys("a!0227581123")

picture=driver.find_element_by_id("CAPTCHA")
# location=picture.location
location={'x':picture.location['x']-395,'y':picture.location['y']+206}
# location={'x':577,"y":326}
# location
left=location['x']
right=location['x']+picture.size['width']
top=location['y']
botton=location['y']+picture.size['height']
from PIL import Image
img=Image.open(r"D:\trydata\screenshot.png")
img=img.crop((left,top,right,botton))
img.save(r"D:\trydata\captcha.png")

# 開始

In [42]:
import os
import base64, re
from io import BytesIO
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import time 
import xl2dict
import smtplib
from email.message import EmailMessage

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
import cv2
import PIL
import numpy
from matplotlib import pyplot as plt

chrome_options = Options()
def base64_to_image(base64_str, image_path=None):
    base64_data = re.sub('^data:image/.+;base64,', '', base64_str)
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    img = Image.open(image_data)
    if image_path:
        img.save(image_path)

def save_captcha():
    driver.switch_to.default_content()
    driver.switch_to.frame(1)
    try:
        driver.execute_script('''
        img = document.getElementById("CAPTCHA");
        var imgCanvas = document.createElement("canvas"),
        imgContext = imgCanvas.getContext("2d");
        imgCanvas.width = img.width;
        imgCanvas.height = img.height;
        imgContext.drawImage(img, 0, 0, imgCanvas.width, imgCanvas.height);
        imgInfom = imgCanvas.toDataURL("image/png");
        localStorage.setItem("imgInfo",imgInfom);
        ''')
        img_data = driver.execute_script('return localStorage.getItem("imgInfo")')
        base64_to_image(img_data,"./captchaDir/CAPTCHA.png")
    except Exception as e:
        print(str(e))

        
if not os.path.exists("captchaDir"):
    os.mkdir("captchaDir")
driver_path = os.path.join(os.getcwd(), 'chromedriver')
driver = webdriver.Chrome(executable_path=driver_path, chrome_options=chrome_options)
driver.get("https://waste.epa.gov.tw/prog/IndexFrame.asp?Func=2")





success=0
while success==0:
    try:
        driver.switch_to.default_content()
    except:
        pass
    driver.switch_to.frame(1)
    driver.page_source
    accountInput=driver.find_element_by_name("fac_user")
    accountInput.clear()
    accountInput.send_keys("J55B6752")
    pwInput=driver.find_element_by_name("fac_pwd")
    pwInput.clear()
    pwInput.send_keys("1121@PINNADA")

    #J55B6752 1121@PINNADA
    #A45B1625 a!0227581123
    ##########################


    captchaText=""
    while len(captchaText)!=4:
        save_captcha()
        mlp=joblib.load(r'D:\trydata\textConsider\model\captcha.pkl')
        # img=Image.open("./captchaDir/CAPTCHA.png")

        pil_image =PIL.Image.open("./captchaDir/CAPTCHA.png").convert("RGB")
        open_cv_image=numpy.array(pil_image)
        plt.imshow(open_cv_image)
        imgray=cv2.cvtColor(open_cv_image,cv2.COLOR_BGR2GRAY)
        ret, thresh=cv2.threshold(imgray,127,255,0)
        image,contours,hierarchy=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnts=sorted([(c,cv2.boundingRect(c)[0]) for c in contours],key=lambda x:x[1])
        ary=[]
        for (c,_) in cnts:
            (x,y,w,h)=cv2.boundingRect(c)
        #         print((x,y,w,h))
            if w>11 and h>=7 and w!=120:
                if x-2>0:
                    ary.append((x-2,y-2,w+4,h+4))
                else:
                    ary.append((x,y,w+4,h+4))
        n=0

        if not os.path.exists("./captchaDir/numPic/"):
            os.mkdir("./captchaDir/numPic/")

        for i in os.listdir("./captchaDir/numPic/"):
            os.remove("./captchaDir/numPic/"+i)

        for (x,y,w,h) in ary:
            fig=plt.figure(figsize=(6, 4), dpi=100) #600*400
            roi=open_cv_image[y:y+h,x:x+w]
            thresh=roi.copy()
        #     ax=fig.add
            plt.imshow(thresh)
            plt.savefig("./captchaDir/numPic/aa_{}.png".format(n),dpi=100)
            n+=1


        data=[]
        basewidth=50
        fig=plt.figure(figsize=(20,20))
        cnt=0
        for idx,img in enumerate(os.listdir("./captchaDir/numPic/")):
            pil_image=PIL.Image.open("./captchaDir/numPic/{}".format(img)).convert('1')
            wpercent=(basewidth/float(pil_image.size[0]))
            hsize=int((float(pil_image.size[1])*float(wpercent)))
            img=pil_image.resize((basewidth,hsize),PIL.Image.ANTIALIAS)
            data.append([pixel for pixel in iter(img.getdata())])
        scaler=StandardScaler()
        scaler.fit(data)
        data_scaled=scaler.transform(data)
        print(mlp.predict(data_scaled))

        captchaText="".join(mlp.predict(data_scaled)).replace("xx","")
        if len(captchaText)!=4:
            driver.find_element_by_link_text("重新載入圖片").click()
            time.sleep(0.5)


    print(captchaText)
    captchaInput=driver.find_element_by_id("securityCode")
    captchaInput.send_keys(captchaText)
    driver.find_element_by_name("Action").click()
    time.sleep(0.5)
    for i in range(5):
        try:
            alert=driver.switch_to_alert()
            alert.accept()
            time.sleep(0.1)
        except:
            pass

    try:
        time.sleep(0.5)
        driver.find_element_by_link_text("越境聯單資料錯誤說明")
        success=1
    except:
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxx")
        time.sleep(0.5)

D:\anaconda\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


['R' 'P' '6' 'C']
RP6C


In [43]:
driver.get("https://waste.epa.gov.tw/prog/stastic.asp?EPC=FAC")
# table=driver.find_element_by_id("table23")
# # table

# for i in table.find_elements_by_css_selector("tr"):
#     if "申報量統計查詢" in i.text:
#         i.find_element_by_css_selector("a").click()
#         break

In [44]:
import datetime
from datetime import timedelta
from selenium.webdriver.support.ui import Select

today = datetime.date.today()
first = datetime.date(day=1, month=today.month, year=today.year)
lastMonthFinal = first - datetime.timedelta(days=1)
lastMonthBegan = datetime.date(day=1,month=lastMonthFinal.month, year=today.year)
yearTaiwan=str(lastMonthFinal.year-1911)

if lastMonthBegan.month>=10:
    lastMonth=str(lastMonthBegan.month)
else:
    lastMonth="0"+str(lastMonthBegan.month)

time.sleep(0.5)
    
yb=Select(driver.find_element_by_name("T3"))
yb.select_by_value(yearTaiwan)

mb=Select(driver.find_element_by_name("T4"))
mb.select_by_value(lastMonth)

db=Select(driver.find_element_by_name("T5"))
db.select_by_value("0"+str(lastMonthBegan.day))


ye=Select(driver.find_element_by_name("T6"))
ye.select_by_value(yearTaiwan)

me=Select(driver.find_element_by_name("T7"))
me.select_by_value(lastMonth)

de=Select(driver.find_element_by_name("T8"))
de.select_by_value(str(lastMonthFinal.day))

driver.find_element_by_name("B1").click()
time.sleep(0.5)

for i in driver.find_elements_by_css_selector("table"):
    for row in i.text.split("\n"):
        if "再利用" in row.split(" "):
            DrownVolume=row.split(" ")[-1]
            print(row)
            print(DrownVolume)#

再利用 0 --- 1 1 0.825
0.825


In [45]:
driver.back()
driver.back()

In [46]:
# driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/table[2]/tbody/tr[2]/td[2]/img")
driver.get("https://waste.epa.gov.tw/prog/stastic.asp?EPC=OilF")
time.sleep(0.5)

In [47]:
yb=Select(driver.find_element_by_name("T3"))
yb.select_by_value(yearTaiwan)

mb=Select(driver.find_element_by_name("T4"))
mb.select_by_value(lastMonth)

db=Select(driver.find_element_by_name("T5"))
db.select_by_value("0"+str(lastMonthBegan.day))


ye=Select(driver.find_element_by_name("T6"))
ye.select_by_value(yearTaiwan)

me=Select(driver.find_element_by_name("T7"))
me.select_by_value(lastMonth)

de=Select(driver.find_element_by_name("T8"))
de.select_by_value(str(lastMonthFinal.day))

driver.find_element_by_name("B1").click()
time.sleep(0.5)

for i in driver.find_elements_by_css_selector("table"):
    for row in i.text.split("\n"):
        if "廢食用油遞送聯單" in row.split(" "):
            oilVolume=row.split(" ")[-1]
            print(row)
            print(oilVolume)#

廢食用油遞送聯單 1 0 1 0 0 1 0.054
0.054


In [48]:
driver.back()
driver.back()

In [49]:
# driver.switch_to.window(driver.window_handles[0])
# driver.switch_to_default_content()
# driver.switch_to_frame(0)
# driver.find_element_by_link_text("基線資料").click()

# driver.switch_to_default_content()
# driver.switch_to_frame(1)
# time.sleep(0.3)
# driver.find_element_by_link_text("廢棄物整體管制號碼設定").click()

# # #這裡部分可能不一樣＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊
# time.sleep(1)
# driver.find_element_by_name("ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderMain$BtnIns").click()
# #這裡部分可能不一樣＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊

# time.sleep(0.5)


# #製造程序
# driver.find_element_by_id("ProduceCode").click()
# driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1]) #driver.window_handles可以列出目前開啟的頁面
# numInput=driver.find_element_by_id("QueryWords")
# numInput.clear()
# numInput.send_keys(str(560001))
# button=driver.find_element_by_xpath("/html/body/div[1]/div[7]/div[2]/input[2]")
# if button.text=="":
#     button.click()
# time.sleep(0.5)
# select=driver.find_element_by_xpath('//*[@id="ResultTable"]/tbody/tr/td[2]')
# if select.text=="餐飲服務作業程序":
#     select.click()
# driver.switch_to.window(driver.window_handles[0])
# driver.switch_to_frame(1)

# #廢棄物代碼
# driver.find_element_by_id("Waste_no").click()
# driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1])
# numInput=driver.find_element_by_id("QueryWords")
# numInput.clear()
# numInput.send_keys("R-0106")
# button=driver.find_element_by_xpath("/html/body/div[1]/div[7]/div[2]/input[2]")
# if button.text=="":
#     button.click()
# time.sleep(0.5)
# select=driver.find_element_by_xpath('//*[@id="ResultTable"]/tbody/tr/td[2]')
# if select.text=="廚餘":
#     select.click()
# driver.switch_to.window(driver.window_handles[0])
# driver.switch_to_frame(1)


# #物種
# driver.find_element_by_id("SpeciesCode").click()
# driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1])
# numInput=driver.find_element_by_id("QueryWords")
# numInput.clear()
# numInput.send_keys("1804")
# button=driver.find_element_by_xpath("/html/body/div[1]/div[7]/div[2]/input[2]")
# if button.text=="":
#     button.click()
# time.sleep(0.5)
# select=driver.find_element_by_xpath('//*[@id="ResultTable"]/tbody/tr/td[2]')
# if select.text=="廚餘":
#     select.click()
# driver.switch_to.window(driver.window_handles[0])
# driver.switch_to_frame(1)

# #物理性質
# driver.find_element_by_id("PhysicalTypeCode").click()
# driver.find_element_by_xpath('//option[@value="P"]').click()
# driver.find_element_by_id("PhysicalTypeCode").click()

# #有害特性
# driver.find_element_by_id("HazardousCharacterCode").click()
# driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1])
# driver.find_element_by_xpath("/html/body/div[1]/div[7]/div[2]/input[2]").click()
# driver.switch_to.window(driver.window_handles[0])
# driver.switch_to_frame(1)

# #主要(有害)成分
# driver.find_element_by_id("HazardousIngredientCode").click()
# driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1])
# driver.find_element_by_xpath("/html/body/div[1]/div[7]/div[2]/input[4]").click()
# driver.switch_to.window(driver.window_handles[0])
# driver.switch_to_frame(1)

# #清理方式
# driver.find_element_by_id("CleanWayCode").click()
# driver.switch_to.window(driver.window_handles[len(driver.window_handles)-1])
# driver.find_element_by_xpath("/html/body/div[1]/div[7]/div[2]/a[2]").click()
# time.sleep(0.5)
# select=driver.find_element_by_xpath('//*[@id="ResultTable"]/tbody/tr[5]/td[2]')
# if select.text=='將廢棄物作為再利用之飼料':
#     select.click()
# driver.switch_to.window(driver.window_handles[0])
# driver.switch_to_frame(1)

# #廢棄物顏色
# driver.find_element_by_id("WColor").send_keys("褐色")

# #平均產生量
# avgQty=0.93 #需改為變數
# avg=driver.find_element_by_id("Avg_Qty")
# avg.clear()
# avg.send_keys(str(avgQty)) 

# #最大產出量
# MaxQty=1 #需改為變數
# Max=driver.find_element_by_id("Max_Qty")
# Max.clear()
# Max.send_keys(str(MaxQty))

# # driver.find_element_by_id("ContentPlaceHolder1_ContentPlaceHolderMain_BtnSave").click() 正式之前不能解開

# 廢油

In [50]:
driver.get("https://waste.epa.gov.tw/massbal/restaurant.asp")

In [51]:
driver.find_element_by_name("app_year").click()
import datetime
today = datetime.date.today()
first = datetime.date(day=1, month=today.month, year=today.year)
lastMonthFinal = first - datetime.timedelta(days=1)
yearTaiwan=str(lastMonthFinal.year-1911)
driver.find_element_by_xpath('//option[@value="' + yearTaiwan + '"]').click()
driver.find_element_by_name("app_year").click()

In [52]:
driver.find_element_by_name("app_month").click()
import datetime
now=datetime.datetime.now()
monthTaiwan=str(now.month-1)
if len(monthTaiwan)==1:
    monthTaiwan="0"+monthTaiwan
driver.find_element_by_xpath('//option[@value="' + lastMonth + '"]').click()
driver.find_element_by_name("app_month").click()

In [53]:
driver.find_elements_by_name("submit")[0].click()

In [54]:
xxxx=100 #廚餘
yyyy=200 #廢油
if xxxx==0 and yyyy==0:
    driver.find_element_by_link_text("本月無產出廢棄物(新設事業亦適用)").click()
    alert=driver.switch_to_alert()
    alert.accept()
    time.sleep(0.1)
else:
    driver.find_element_by_link_text("事業廢棄物產出種類、數量").click()
    time.sleep(0.5)
    
    
    #這裡部分可能不一樣************************
    for i in driver.find_elements_by_css_selector("tbody"):
        for j in i.find_elements_by_css_selector("tr"):
            if "泥狀" in j.text:
                productInput=j.find_element_by_id("app_qty")
                productInput.clear()
                productInput.send_keys(str(xxxx))
                includeWaterInput=j.find_element_by_id("inwater")
                includeWaterInput.clear()
                includeWaterInput.send_keys("0")
                j.find_element_by_id("chkWasteItem").click()
            if "液狀" in j.text:
                productInput=j.find_element_by_id("app_qty")
                productInput.clear()
                productInput.send_keys(str(yyyy))
                includeWaterInput=j.find_element_by_id("inwater")
                includeWaterInput.clear()
                includeWaterInput.send_keys("0")
                j.find_element_by_id("chkWasteItem").click()
    #這裡部分可能不一樣************************
    # driver.find_element_by_name("B1").click()
#     alert=driver.switch_to_alert()
#     alert.accept()
#     time.sleep(0.1)

In [55]:
# import pandas as pd

In [56]:
# import sys
# df=pd.read_excel(r"C:\Users\andy.yuan\Desktop\食安自動化\需要產出的資料格式.xlsx", encoding=sys.getfilesystemencoding())

In [41]:
# data=df.T.to_dict()
# for i in range(len(df)-1):
#     print(data[i]["帳號"])
#     print(data[i]["密碼"])
#     print(data[i]["本月廢油產出量"])
#     print("="*50)

In [ ]:
#################
#               #
#               #
#               #
#################

In [ ]:
###